In [25]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import time

In [38]:
#load the balanced data
data_balanced = pd.read_csv('UNSW_NB15_balanced_smote.csv')

### LSTM Model for Multi-class Classification

In [41]:
# train an lstm model on the data
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn import model_selection
from sklearn import metrics
import tensorflow as tf
X = data_balanced.drop(['attack_cat','label'], axis=1)
Y = data_balanced[['attack_cat']]
Y['attack_cat'] = Y['attack_cat'].astype('int')
# Y['label'] = Y['label'].astype('int')
# one hot encode attack_cat
Y = pd.get_dummies(Y,columns=['attack_cat'])

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

# reshape the data for LSTM
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

# convert all to numpy arrays
X_train = np.asarray(X_train).astype(np.float32)
Y_train = np.asarray(Y_train).astype(np.int32)
X_test = np.asarray(X_test).astype(np.float32)
Y_test = np.asarray(Y_test).astype(np.int32)

print(X_train.shape)
# define the model
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation='relu'))
model.add(Dropout(0.1))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(Dropout(0.1))
model.add(LSTM(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(6, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
start = time.time()
# train the model
history = model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test), shuffle=True)
end = time.time()
print("Time taken to train the model: ", end - start)

C:\Users\Ameya\AppData\Local\Temp\ipykernel_30200\5757800.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y['attack_cat'] = Y['attack_cat'].astype('int')


(446400, 1, 196)
Epoch 1/10
13950/13950 [==============================] - 31s 2ms/step - loss: 0.1657 - accuracy: 0.7833 - val_loss: 0.1418 - val_accuracy: 0.8121
Epoch 2/10
13950/13950 [==============================] - 31s 2ms/step - loss: 0.1396 - accuracy: 0.8160 - val_loss: 0.1337 - val_accuracy: 0.8224
Epoch 3/10
13950/13950 [==============================] - 30s 2ms/step - loss: 0.1352 - accuracy: 0.8205 - val_loss: 0.1298 - val_accuracy: 0.8248
Epoch 4/10
13950/13950 [==============================] - 30s 2ms/step - loss: 0.1323 - accuracy: 0.8231 - val_loss: 0.1277 - val_accuracy: 0.8276
Epoch 5/10
13950/13950 [==============================] - 32s 2ms/step - loss: 0.1306 - accuracy: 0.8249 - val_loss: 0.1277 - val_accuracy: 0.8285
Epoch 6/10
13950/13950 [==============================] - 30s 2ms/step - loss: 0.1294 - accuracy: 0.8266 - val_loss: 0.1262 - val_accuracy: 0.8296
Epoch 7/10
13950/13950 [==============================] - 30s 2ms/step - loss: 0.1283 - accuracy: 0.8

In [42]:
# print model classifaciton report
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
# y_pred = np.argmax(y_pred, axis=1)
print('Classification Report')
print(metrics.classification_report(Y_test, y_pred))
print('Accuracy:')
print(metrics.accuracy_score(Y_test, y_pred))


3488/3488 [==============================] - 4s 1ms/step
Classification Report
              precision    recall  f1-score   support

           0       0.67      0.89      0.77     18639
           1       0.89      0.60      0.72     18534
           2       0.78      0.83      0.80     18672
           3       1.00      0.97      0.99     18497
           4       0.99      0.73      0.84     18721
           5       0.94      0.81      0.87     18537

   micro avg       0.86      0.81      0.83    111600
   macro avg       0.88      0.81      0.83    111600
weighted avg       0.88      0.81      0.83    111600
 samples avg       0.81      0.81      0.81    111600

Accuracy:
0.8061290322580645


C:\Users\Ameya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [43]:
# print latency for model prediction
start = time.time()
y_pred = model.predict(X_test)
end = time.time()
print('Average Latency for model prediction:(ms)')
print((end-start)/len(y_pred)*1000)

3488/3488 [==============================] - 4s 1ms/step
Average Latency for model prediction:(ms)
0.04159534062963233
